In [1]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

# Latent Dirichlet Allocation

LSI pozwala w pewnym sensie znajdowac kluczowe tematy w tekście i dla danego dokumentu określać najbliższy mu temat. Jest to podejście geometryczne. 

LDA jest podejściem probabilistycznym do modelowania tematów. Jest bardziej dokładny choć wolniejszy.

# Model LDA - Latent Dirichlet Allocation (ukryta alokacja Dirichleta)


Motywacja: przedstawienie tekstu jako mieszanki tematów.


Temat - rozkład prawdopodobieństwa na zbiorze słów.


Przykład:
*  <s>Mam</s> gorączkę <s>i</s> katar.
* Graliśmy <s>w</s> siatkówkę.
* Grając <s>w</s> piłkę, wzmacniamy organizm.


Ile "tematów" widzimy?
<br>

<br>

<br>

<br>

<br>

<br>

Intuicyjnie: dwa tematy: "sport" oraz "zdrowie".
* Pierwsze zdanie = 100% zdrowie
* Drugie zdanie = 100% sport
* Trzecie zdanie = 50% sport + 50% zdrowie

LDA to model probabilistyczny, który wykorzystuje dwie wartości prawdopodobieństwa: 

* P (słowo | tematy) 
* P (tematy | dokumenty) 

do stworzenia klastrów.


Proces budowy modelu LDA zakłada więc:
* identyfikację tematów reprezentowanych przez dokumenty korpusu,
* oszacowanie rozkładu prawdopodobieństw wystąpienia wyrazów dla każdego z tematów (stosowany jest rozkład Dirichleta),
* oszacowanie rozkładu prawdopodobieństw wystąpienia tematów w rozpatrywanych dokumentach (stosowany jest również rozkład Dirichleta).

Wykorzystując metodę identyfikacji słów kluczowych opartą na ukrytej alokacji Dirichleta można:
* dla każdego dokumentu obliczyć prawdopodobieństwo wystąpienia w nim każdego z tematów;
* dla każdego tematu obliczyć prawdopodobieństwo wystąpienia słów;
* obliczyć prawdopodobieństwa wystąpienia poszczególnych słów w dokumencie (jako iloczyny dwóch wyżej wymienionych prawdopodobieństw) 

# Generatywność

Mając prawdopodobieństwa $p_1 = P (słowo | tematy) $, $p_2 = P (tematy | dokumenty)$ możemy wygeneraować dokument:

- wybieramy (losujemy) prawdopodobieństwo przynależności dokumentu do tematu (z $p_1$) - dokuemnt zawsze należy do wielu tematów

- generujemy słowa w dokumencie - najpierw losujemy do jakiego tematu należy słowo, a potem generujemy słowo z tego tematu (z $p_2$)

# Uczenie

1. Idziemy przez wszystkie słowa i wszystkie dokumentu i losowo przyporządkowujemy je do tematów.
2. Iteracyjnie idzemy: bierzemy dokuemnt $d$  i słowo $w$ i przyporządkowujemy je do najlepiej pasującego tematu - maksymalizującego $P (słowo | tematy) 
* P (tematy | dokumenty) $
3. Po przejsciu przez wszystkie dokumenty aktualizujemy przydział słów do tematów i tematów do dokumentów i wracamy do punktu 2

# Zad

Na początek wczytujemy korpus z dysku. Użyjemy przykład stworzonego w poprzednim notebook-u.

In [2]:
# documents = ["Romeo and Juliet",
#          "Juliet: O happy dagger",
#          "Romeo died by dagger",
#          "'Live free or die', that’s the New-Hampshire’s motto",
#          "Did you know, New-Hampshire is in New-England"]

In [3]:
if (os.path.exists("tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
    corpus = corpora.MmCorpus('tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")



Used files generated from first tutorial


In [4]:
for d in corpus:
    print(d)

[(0, 1.0), (1, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [5]:
scipy_csc_matrix = matutils.corpus2csc(corpus)
print(scipy_csc_matrix.todense())

[[1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [6]:
print(dictionary.token2id)

{'juliet': 0, 'romeo': 1, 'dagger': 2, 'happy': 3, 'juliet:': 4, 'o': 5, 'by': 6, 'died': 7, "'live": 8, "die',": 9, 'free': 10, 'motto': 11, 'new-hampshire’s': 12, 'or': 13, 'that’s': 14, 'did': 15, 'is': 16, 'know,': 17, 'new-england': 18, 'new-hampshire': 19, 'you': 20}


# Budujemy model LDA
Budujemy model LDA i transformujemy dane

* **num_topics=2** oznacza ilość modelowanych tematów

In [7]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = model[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for d in corpus_lda:
    print(d)

[(0, 0.81393474), (1, 0.18606526)]
[(0, 0.8889627), (1, 0.11103739)]
[(0, 0.8917853), (1, 0.10821463)]
[(0, 0.07954125), (1, 0.9204588)]
[(0, 0.07649173), (1, 0.9235082)]


### Dla każdego dokumentu dostajemy prawdopodobieństwo przynależności dokumentu do danego tematu.

Możemy też zobaczyć z czego składają się tematy:

In [8]:
model.show_topics()

[(0,
  '0.105*"dagger" + 0.097*"romeo" + 0.066*"died" + 0.063*"by" + 0.062*"o" + 0.061*"juliet:" + 0.060*"juliet" + 0.059*"happy" + 0.041*"\'live" + 0.040*"that’s"'),
 (1,
  '0.063*"is" + 0.063*"know," + 0.063*"you" + 0.063*"did" + 0.062*"new-hampshire" + 0.062*"new-england" + 0.056*"new-hampshire’s" + 0.055*"or" + 0.053*"die\'," + 0.052*"free"')]

In [9]:
model.print_topics(2)

[(0,
  '0.105*"dagger" + 0.097*"romeo" + 0.066*"died" + 0.063*"by" + 0.062*"o" + 0.061*"juliet:" + 0.060*"juliet" + 0.059*"happy" + 0.041*"\'live" + 0.040*"that’s"'),
 (1,
  '0.063*"is" + 0.063*"know," + 0.063*"you" + 0.063*"did" + 0.062*"new-hampshire" + 0.062*"new-england" + 0.056*"new-hampshire’s" + 0.055*"or" + 0.053*"die\'," + 0.052*"free"')]

In [10]:
model.print_topics(num_topics=2, num_words=4)

[(0, '0.105*"dagger" + 0.097*"romeo" + 0.066*"died" + 0.063*"by"'),
 (1, '0.063*"is" + 0.063*"know," + 0.063*"you" + 0.063*"did"')]

# Zad

Chcemy posortować słowa każdego tematu i wybrać 5 najważniejszych - co można powiedzieć o tematach?

Proszę zobaczyć na funkcje typu get_topics(), get_term_topics(...): https://radimrehurek.com/gensim/models/ldamodel.html

In [11]:
import numpy as np
topics = np.argsort(model.get_topics()[0,:])[::-1] #::-1 sortowanie w odwrotnej kolejności
for x in topics[:5]:
    print(dictionary[x])

dagger
romeo
died
by
o


In [12]:
model.get_topic_terms(topicid=0)

[(2, 0.10525187),
 (1, 0.09694249),
 (7, 0.06611413),
 (6, 0.0630177),
 (5, 0.062458374),
 (4, 0.060585562),
 (0, 0.06049739),
 (3, 0.058546674),
 (8, 0.041037835),
 (14, 0.040186286)]

In [13]:
topic2_terms = model.get_topic_terms(topicid=0)
topic2_words = [
    (dictionary.get(i), j)
    for i,j in topic2_terms
]
topic2_words

[('dagger', 0.10525187),
 ('romeo', 0.09694249),
 ('died', 0.06611413),
 ('by', 0.0630177),
 ('o', 0.062458374),
 ('juliet:', 0.060585562),
 ('juliet', 0.06049739),
 ('happy', 0.058546674),
 ("'live", 0.041037835),
 ('that’s', 0.040186286)]

In [14]:
[
    (dictionary.get(i), j)
    for i,j in model.get_topic_terms(topicid=0)
]

[('dagger', 0.10525187),
 ('romeo', 0.09694249),
 ('died', 0.06611413),
 ('by', 0.0630177),
 ('o', 0.062458374),
 ('juliet:', 0.060585562),
 ('juliet', 0.06049739),
 ('happy', 0.058546674),
 ("'live", 0.041037835),
 ('that’s', 0.040186286)]

# Zad 

Proszę posortować zdania najbardziej pasujące do danego tematu. 

In [15]:
import gensim

top_inex = 0

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    
#trzeba by wypisać raczej zdania niż ich reprezentacje bag-of-words, ale tu nie mam dostepu do tekstu

[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(0, 1.0), (1, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [16]:
top_inex = 1

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    

[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(0, 1.0), (1, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]


In [17]:
documents = ["Romeo and Juliet",
         "Juliet: O happy dagger",
         "Romeo died by dagger",
         "'Live free or die', that’s the New-Hampshire’s motto",
         "Did you know, New-Hampshire is in New-England"]

top_inex = 0

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort( numpy_corpus[top_inex,:] )[::-1]
for x in docs[:5]:
    print(documents[x])
    

Romeo died by dagger
Juliet: O happy dagger
Romeo and Juliet
'Live free or die', that’s the New-Hampshire’s motto
Did you know, New-Hampshire is in New-England


In [18]:
top_inex = 1

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(documents[x])
    

Did you know, New-Hampshire is in New-England
'Live free or die', that’s the New-Hampshire’s motto
Romeo and Juliet
Juliet: O happy dagger
Romeo died by dagger


# Zad. 
Sprawdzić do jakiego tematu pasuje nowy dokument i jakie są mu najbliższe

In [19]:
doc = "died dagger"

In [20]:
doc_rep = dictionary.doc2bow(doc.split(' '))
# print(doc_rep)
doc_assignments = model[doc_rep]
print(doc_assignments)

[(0, 0.8227634), (1, 0.17723662)]


In [21]:
index = similarities.MatrixSimilarity(corpus_lda)

sims = index[doc_assignments]
print(list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(0, 0.9999208), (1, 0.9961348), (2, 0.9958265), (3, 0.29393426), (4, 0.2905712)]
[(0, 0.9999208), (1, 0.9961348), (2, 0.9958265), (3, 0.29393426), (4, 0.2905712)]


# Wizualizacja modelu LDA:

pyLDAvis

http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb


* Czerwone słupki reprezentują częstotliwość słów w danym temacie (proporcjonalnie do $P (słowo \ | \ tematy) $), 
* Niebieskie słupki reprezentują częstotliwość tematów w całym korpusie (proporcjonalnie do $P(tematy \ | \ dokumenty)$. 

Zmień wartość $\lambda$, aby dostosować rankingi słów: 
 * małe wartości $\lambda$ (blisko 0) podkreślają potencjalnie rzadkie, ale ekskluzywne słowa dla wybranego tematu
 * duże wartości $\lambda$ (blisko 1) podkreślają częste, ale niekoniecznie ekskluzywne słowa dla wybranego tematu. 
 
W http://www.kennyshirley.com/LDAvis/ sugeruje się żeby ustawiać $\lambda$ w pobliżu 0.6. Podobno pomaga to użytkownikom w interpretacji tematu.

In [22]:
import pyLDAvis.gensim

In [23]:
pyLDAvis.enable_notebook()

In [24]:
# pyLDAvis.gensim.prepare??

In [25]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

C:\Users\admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.029658  0.0       1        1  57.536007
0     -0.029658  0.0       2        1  42.463993, topic_info=     Category      Freq             Term     Total  loglift  logprob
term                                                                
2     Default  1.000000           dagger  1.000000  21.0000  21.0000
1     Default  1.000000            romeo  1.000000  20.0000  20.0000
7     Default  0.000000             died  0.000000  19.0000  19.0000
6     Default  1.000000               by  1.000000  18.0000  18.0000
5     Default  1.000000                o  1.000000  17.0000  17.0000
4     Default  1.000000          juliet:  1.000000  16.0000  16.0000
0     Default  1.000000           juliet  1.000000  15.0000  15.0000
3     Default  1.000000            happy  1.000000  14.0000  14.0000
16    Default  1.000000               is  1.000000  13.0000  13.0000
17    Default  1.000000            know,  1.000000  12.0000  12.0000
20    Default  1.000000              you  1.000000  11.0000  11.0000
15    Default  1.000000              did  1.000000  10.0000  10.0000
19    Default  1.000000    new-hampshire  1.000000   9.0000   9.0000
18    Default  1.000000      new-england  1.000000   8.0000   8.0000
12    Default  1.000000  new-hampshire’s  1.000000   7.0000   7.0000
13    Default  1.000000               or  1.000000   6.0000   6.0000
8     Default  1.000000            'live  1.000000   5.0000   5.0000
14    Default  1.000000           that’s  1.000000   4.0000   4.0000
9     Default  1.000000            die',  1.000000   3.0000   3.0000
10    Default  1.000000             free  1.000000   2.0000   2.0000
11    Default  1.000000            motto  1.000000   1.0000   1.0000
16     Topic1  0.837864               is  1.094958   0.2851  -2.7596
17     Topic1  0.836473            know,  1.094667   0.2837  -2.7613
20     Topic1  0.835180              you  1.094396   0.2824  -2.7628
15     Topic1  0.833687              did  1.094083   0.2809  -2.7646
19     Topic1  0.824035    new-hampshire  1.092062   0.2712  -2.7763
18     Topic1  0.814892      new-england  1.090147   0.2617  -2.7874
12     Topic1  0.740352  new-hampshire’s  1.074534   0.1802  -2.8834
13     Topic1  0.728845               or  1.072125   0.1668  -2.8990
9      Topic1  0.695271            die',  1.065092   0.1262  -2.9462
...       ...       ...              ...       ...      ...      ...
8      Topic1  0.656078            'live  1.056883   0.0760  -3.0042
3      Topic1  0.439769            happy  1.011577  -0.2803  -3.4042
0      Topic1  0.415669           juliet  1.006530  -0.3316  -3.4606
4      Topic1  0.414579          juliet:  1.006302  -0.3340  -3.4632
5      Topic1  0.391443                o  1.001456  -0.3866  -3.5207
6      Topic1  0.384532               by  1.000008  -0.4030  -3.5385
1      Topic1  0.546948            romeo  1.493758  -0.4519  -3.1861
7      Topic1  0.346279             died  0.991997  -0.4997  -3.6432
2      Topic1  0.444294           dagger  1.472259  -0.6453  -3.3940
2      Topic2  1.027965           dagger  1.472259   0.4973  -2.2514
7      Topic2  0.645718             died  0.991997   0.4272  -2.7164
1      Topic2  0.946810            romeo  1.493758   0.4006  -2.3336
6      Topic2  0.615476               by  1.000008   0.3711  -2.7643
5      Topic2  0.610013                o  1.001456   0.3608  -2.7733
4      Topic2  0.591722          juliet:  1.006302   0.3255  -2.8037
0      Topic2  0.590861           juliet  1.006530   0.3238  -2.8052
3      Topic2  0.571809            happy  1.011577   0.2861  -2.8379
8      Topic2  0.400805            'live  1.056883  -0.1131  -3.1933
14     Topic2  0.392488           that’s  1.059087  -0.1361  -3.2142
11     Topic2  0.376071            motto  1.063436  -0.1830  -3.2570
10     Topic2  0.371515             free  1.064643  -0.1963  -3.2691
9      Topic2  0.369821            